In [1]:
# 以下是特別為了進行cross validation切割以及彙整所有CT資料使用，請視情況使用，無法直接run。
import glob, pandas as pd, os
import numpy as np
from pathlib import Path
from sklearn.model_selection import StratifiedKFold

In [2]:
#取得路徑下所有資料
count=0
data_getcwd = '/ssd8/2023COVID19/' #base path
full_data_list = [] #儲存所有slice清單
full_path_list = [] #儲存所有CT資料夾清單

# for i in ['negative', 'positive']:
#         for path in Path(os.path.join(data_getcwd,'Train_Valid_dataset/train/{}').format(i)).rglob('*.jpg'): #尋找所有*jpg檔案
#                 ct_name = str(path.parent).split('/')[-1]
#                 label = 0 if 'negative' in str(path) else 1
#                 token_key = ct_name+'_0' if 'negative' in str(path) else ct_name+'_1'
#                 full_data_list.append([str(path.parent), ct_name, str(path.name), label, 0, token_key])
#                 full_path_list.append([str(path.parent), label, 0, token_key])     

# for path in Path(os.path.join(data_getcwd,'Unzip_dataset/train/negative/extra_challenge1_non_cov_train_new_icassp')).rglob('*.jpg'): #尋找challenge所有*jpg檔案
#         ct_name = str(path.parent).split('/')[-1]
#         label = 0 if 'negative' in str(path) else 1
#         token_key = ct_name+'_0_cha' if 'negative' in str(path) else ct_name+'_1_cha' #用於區分covid和non-covid中重複的資料夾名稱，之後也會以這個columns作為篩選key
#         full_data_list.append([str(path.parent), ct_name, str(path.name), label, 0, token_key])
#         full_path_list.append([str(path.parent), label, 0, token_key])



for i in ['negative', 'positive']:
        for path in Path(os.path.join(data_getcwd,'Train_Valid_dataset/valid/{}').format(i)).rglob('*.jpg'): #尋找所有*jpg檔案
                ct_name = str(path.parent).split('/')[-1]
                label = 0 if 'negative' in str(path) else 1
                token_key = ct_name+'_0' if 'negative' in str(path) else ct_name+'_1'
                full_data_list.append([str(path.parent), ct_name, str(path.name), label, 0, token_key])
                full_path_list.append([str(path.parent), label, 0, token_key])

for path in Path(os.path.join(data_getcwd,'Unzip_dataset/valid/negative/extra_challenge1_non_cov_val_new_icassp')).rglob('*.jpg'): #尋找challenge所有*jpg檔案
        ct_name = str(path.parent).split('/')[-1]
        label = 0 if 'negative' in str(path) else 1
        token_key = ct_name+'_0_cha' if 'negative' in str(path) else ct_name+'_1_cha' #用於區分covid和non-covid中重複的資料夾名稱，之後也會以這個columns作為篩選key
        full_data_list.append([str(path.parent), ct_name, str(path.name), label, 0, token_key])
        full_path_list.append([str(path.parent), label, 0, token_key])

for path in Path(os.path.join(data_getcwd,'Unzip_dataset/valid/positive/extra_challenge1_cov_val_new_icassp')).rglob('*.jpg'): #尋找challenge所有*jpg檔案
        ct_name = str(path.parent).split('/')[-1]
        label = 0 if 'negative' in str(path) else 1
        token_key = ct_name+'_0_cha' if 'negative' in str(path) else ct_name+'_1_cha' #用於區分covid和non-covid中重複的資料夾名稱，之後也會以這個columns作為篩選key
        full_data_list.append([str(path.parent), ct_name, str(path.name), label, 0, token_key])
        full_path_list.append([str(path.parent), label, 0, token_key])

full_data_list = pd.DataFrame(full_data_list, columns=['path', 'ct_name', 'slice_name','label', 'fold', 'token_key']).reset_index(drop=True) #重置index用於篩選location
full_path_list = pd.DataFrame(full_path_list, columns=['parent_path', 'label', 'fold', 'token_key']).drop_duplicates(subset='parent_path').reset_index(drop=True) #重置index用於篩選location

In [3]:
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

In [4]:

for idx, (train, test) in enumerate(kfold.split(full_path_list['token_key'].values, full_path_list['label'].values)):#基於index設立fold number
    # print(test)
    full_path_list.loc[test, 'fold']=idx+1#將挑選出來的fold index用於篩選CT資料夾清單的index資料，並且將fold欄位全覆蓋為當前的fold number

In [5]:
from tqdm import tqdm
for i in tqdm(range (len(full_path_list))): #根據CT資料夾清單對所有Slice進行fold欄位覆蓋
    ct_name = full_path_list.loc[i].token_key#取得CT資料夾名字
    row_index = full_data_list['token_key']==ct_name #取得所有Slice清單中符合當前CT資料夾名字的index
    fold = full_path_list.loc[i].fold #取得CT資料夾清單的fold數字
    full_data_list.loc[row_index,'fold'] = fold #將挑選出來的slice清單的fold數字，全部替換成CT資料夾清單挑選出來的fold數字

100%|██████████| 704/704 [00:12<00:00, 58.10it/s]


In [6]:
drop_valid_df_list_ = full_data_list.drop_duplicates(subset='token_key')
all_train_list =[drop_valid_df_list_[drop_valid_df_list_['label']==1].path.values.tolist(),
                 drop_valid_df_list_[drop_valid_df_list_['label']==0].path.values.tolist()]

In [7]:
for i in [1,2,3,4]:
    temp = full_data_list[full_data_list['fold']==i]
    folder = temp['token_key'].drop_duplicates()
    label = temp.drop_duplicates(subset='token_key').label
    print("Fold-{} CT number: {} || Slice Sample: {} || CT covid: {}; CT non-covid: {}".format(i, len(folder),len(temp), sum(label), len(label)-sum(label)))
# full_data_list_save = full_data_list[['path', 'ct_name', 'slice_name','label', 'fold']]
full_data_list_save = full_data_list
full_data_list_save.to_csv('./Train_Valid_dataset/chih_4_fold_covid_valid_df.csv', index=False, encoding='utf-8-sig')
# full_data_list_save.to_csv('./Train_Valid_dataset/chih_4_fold_covid_train_df.csv', index=False, encoding='utf-8-sig')


Fold-1 CT number: 176 || Slice Sample: 37707 || CT covid: 58; CT non-covid: 118
Fold-2 CT number: 176 || Slice Sample: 37101 || CT covid: 59; CT non-covid: 117
Fold-3 CT number: 176 || Slice Sample: 39555 || CT covid: 59; CT non-covid: 117
Fold-4 CT number: 176 || Slice Sample: 37544 || CT covid: 59; CT non-covid: 117


In [ ]:
235